In [2]:
import os
import json
import hydra
import numpy as np
import enreg.tools.general as g
import mplhep as hep
import awkward as ak
import matplotlib.pyplot as plt
from omegaconf import DictConfig

In [3]:
hep.style.use(hep.styles.CMS)

In [4]:
data_zh = g.load_all_data( "/home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau", n_files=100)

data_zz = g.load_all_data( "/home/laurits/ENREG/ntuples/20240423_PV_test/Z_Ztautau", n_files=100)


output_dir = os.path.join("/home/norman/pT_eta_plots")


os.makedirs(output_dir, exist_ok=True)



[1/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_200254.parquet
[2/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_203213.parquet
[3/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_204507.parquet
[4/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_201924.parquet
[5/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_201761.parquet
[6/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_203635.parquet
[7/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_203654.parquet
[8/100] Loading from /home/laurits/ENREG/ntuples/20240402_full_stats/ZH_Htautau/reco_p8_ee_ZH_Htautau_ecm380_202316.parquet
[9/100] 

In [4]:
gen_tau_pt_zz = g.reinitialize_p4(data_zz["gen_jet_full_tau_p4s"]).pt
reco_tau_pt_zz = g.reinitialize_p4(data_zz["event_reco_cand_p4s"]).pt

gen_jet_pt_zz = g.reinitialize_p4(data_zz["gen_jet_p4s"]).pt
reco_jet_pt_zz = g.reinitialize_p4(data_zz["reco_jet_p4s"]).pt

In [89]:
gen_tau_pt_zh = g.reinitialize_p4(data_zh["gen_jet_full_tau_p4s"]).pt
reco_tau_pt_zh = g.reinitialize_p4(data_zh["event_reco_cand_p4s"]).pt

gen_jet_pt_zh = g.reinitialize_p4(data_zh["gen_jet_p4s"]).pt
reco_jet_pt_zh = g.reinitialize_p4(data_zh["reco_jet_p4s"]).pt

In [90]:
def plot_pt(values1, values2, label1, label2, title, xmin, xmax, output_path, nbins=30):
    fig, ax = plt.subplots()
    bins = np.linspace(xmin, xmax, num=30)
    hist1, bin_edges = np.histogram(values1, bins=bins)
    hist1 = hist1 / ak.sum(hist1)
    hep.histplot(hist1, bin_edges, density = True, label = label1)
    hist2, bin_edges = np.histogram(values2, bins=bins)
    hist2 = hist2 / ak.sum(hist2)
    hep.histplot(hist2, bin_edges, density = True, label = label2)
    plt.xlabel("GeV", loc="center")
    plt.xlim((xmin, xmax))
    plt.yscale('log')
    plt.title(title)
    plt.legend(loc="upper right")
    pT_mean_zz, pT_stdev_zz = round(np.mean(values1),2 ), round(np.std(values1), 2)
    pT_mean_zh, pT_stdev_zh = round(np.mean(values2),2 ), round(np.std(values2), 2)
    textstr1 = f"Mean zz={'{:1}'.format(pT_mean_zz)} \nStdev zz={'{:1}'.format(pT_stdev_zz)}"
    textstr2 = f"Mean zh={'{:1}'.format(pT_mean_zh)} \nStdev zh={'{:1}'.format(pT_stdev_zh)}"
    props = {"boxstyle": "round", "facecolor": "white", "alpha": 0.5}
    ax.text(0.6, 0.8, textstr1, transform=ax.transAxes, fontsize=16, verticalalignment="top", bbox=props)
    ax.text(0.6, 0.7, textstr2, transform=ax.transAxes, fontsize=16, verticalalignment="top", bbox=props)
    plt.savefig(output_path)
    plt.close("all")

In [79]:
# Gen Tau pT
plot_pt(
    values1=ak.flatten(gen_tau_pt_zz, axis=-1),
    values2=ak.flatten(gen_tau_pt_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Gen Jet Tau pT',
    xmin=0,
    xmax=380,
    output_path=os.path.join(output_dir, "gen_jet_tau_pt.png"),
)
# Gen Jet pT
plot_pt(
    values1=ak.flatten(gen_jet_pt_zz, axis=-1),
    values2=ak.flatten(gen_jet_pt_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Gen Jet pT',
    xmin=0,
    xmax=380,
    output_path=os.path.join(output_dir, "gen_jet_pt.png"),
)
# Reco Jet pT
plot_pt(
    values1=ak.flatten(reco_jet_pt_zz, axis=-1),
    values2=ak.flatten(reco_jet_pt_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Reco Jet pT',
    xmin=0,
    xmax=380,
    output_path=os.path.join(output_dir, "reco_jet_pt.png"),
)


In [91]:
# Reco cand pT idk seems fishy, probably needs to be genmatched and then plotted
plot_pt(
    values1=ak.flatten(reco_tau_pt_zz, axis=-1),
    values2=ak.flatten(reco_tau_pt_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Reco cand pT',
    xmin=0,
    xmax=380,
    output_path=os.path.join(output_dir, "reco_cand_pt.png"),
)

In [97]:
gen_tau_eta_zz = g.reinitialize_p4(data_zz["gen_jet_full_tau_p4s"]).eta
reco_tau_eta_zz = g.reinitialize_p4(data_zz["event_reco_cand_p4s"]).eta

gen_jet_eta_zz = g.reinitialize_p4(data_zz["gen_jet_p4s"]).eta
reco_jet_eta_zz = g.reinitialize_p4(data_zz["reco_jet_p4s"]).eta

In [98]:
gen_tau_eta_zh = g.reinitialize_p4(data_zh["gen_jet_full_tau_p4s"]).eta
reco_tau_eta_zh = g.reinitialize_p4(data_zh["event_reco_cand_p4s"]).eta

gen_jet_eta_zh = g.reinitialize_p4(data_zh["gen_jet_p4s"]).eta
reco_jet_eta_zh = g.reinitialize_p4(data_zh["reco_jet_p4s"]).eta

In [116]:
def plot_eta(values1, values2, label1, label2, title, xmin, xmax, output_path, nbins=30):
    fig, ax = plt.subplots()
    bins = np.linspace(xmin, xmax, num=30)
    hist1, bin_edges = np.histogram(values1, bins=bins)
    hist1 = hist1 / ak.sum(hist1)
    hep.histplot(hist1, bin_edges, density = True, label = label1)
    hist2, bin_edges = np.histogram(values2, bins=bins)
    hist2 = hist2 / ak.sum(hist2)
    hep.histplot(hist2, bin_edges, density = True, label = label2)
    plt.xlabel("au", loc="center")
    plt.xlim((xmin, xmax))
    plt.yscale('log')
    plt.title(title)
    plt.legend(loc="upper right")
    eta_mean_zz = round(np.mean(values1),2 )
    eta_mean_zh = round(np.mean(values2),2 )
    textstr1 = f"Mean zz={'{:1}'.format(eta_mean_zz)}"
    textstr2 = f"Mean zh={'{:1}'.format(eta_mean_zh)}"
    props = {"boxstyle": "round", "facecolor": "white", "alpha": 0.5}
    ax.text(0.05, 0.95, textstr1, transform=ax.transAxes, fontsize=16, verticalalignment="top", bbox=props)
    ax.text(0.05, 0.85, textstr2, transform=ax.transAxes, fontsize=16, verticalalignment="top", bbox=props)
    plt.savefig(output_path)
    plt.close("all")

In [117]:
# Gen Tau eta
plot_eta(
    values1=ak.flatten(gen_tau_eta_zz, axis=-1),
    values2=ak.flatten(gen_tau_eta_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Gen Jet Tau eta',
    xmin=-5,
    xmax=5,
    output_path=os.path.join(output_dir, "gen_jet_tau_eta.png"),
)
# Gen Jet eta
plot_eta(
    values1=ak.flatten(gen_jet_eta_zz, axis=-1),
    values2=ak.flatten(gen_jet_eta_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Gen Jet eta',
    xmin=-5,
    xmax=5,
    output_path=os.path.join(output_dir, "gen_jet_eta.png"),
)
# Reco Jet eta
plot_eta(
    values1=ak.flatten(reco_jet_eta_zz, axis=-1),
    values2=ak.flatten(reco_jet_eta_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Reco Jet eta',
    xmin=-5,
    xmax=5,
    output_path=os.path.join(output_dir, "reco_jet_eta.png"),
)

In [118]:
# reco Tau eta
plot_eta(
    values1=ak.flatten(reco_tau_eta_zz, axis=-1),
    values2=ak.flatten(reco_tau_eta_zh, axis=-1),
    label1 = 'ZZ',
    label2 = 'ZH',
    title = 'Reco cand eta',
    xmin=-5,
    xmax=5,
    output_path=os.path.join(output_dir, "reco_cand_eta.png"),
)